In [1]:

from collections import Counter

import numpy as np


import torch

from datasets import (

    load_from_disk,
)
from transformers import (
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)


import pickle

from sklearn.metrics import classification_report
from transformers import BertTokenizerFast, BertForTokenClassification

from evaluate import load as load_metric

In [2]:
model_name = "bert-base-cased"

In [3]:
conll_main = load_from_disk("./splits/conll_main")

ontonotes_main = load_from_disk("./splits/ontonotes_main")

ontonotes_nw_encoded = load_from_disk("./splits/ontonotes_newswire")
ontonotes_rest_encoded = load_from_disk("./splits/ontonotes_rest")

In [22]:
from datasets import Dataset

# Reverse map: int → string label
id2label = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG',
            5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}

# Define a function to map the list of integer labels to strings


def map_labels_to_names(example):
    example['label_names'] = [id2label[label] for label in example['labels']]
    return example


# Apply to entire dataset
conll_main = conll_main.map(map_labels_to_names)

Map:   0%|          | 0/16595 [00:00<?, ? examples/s]

In [25]:
def map_ontonotes_labels(example):
    example['label_names'] = [ontonotes_id_to_label[i]
                              for i in example['labels']]
    return example


# Apply to dataset
ontonotes_main = ontonotes_main.map(map_ontonotes_labels)

Map:   0%|          | 0/61371 [00:00<?, ? examples/s]

In [44]:
# Replace 'your_file.pkl' with your actual path
with open('results/results/bert-base-cased_conll_results.pkl', 'rb') as f:
    onto_pred = pickle.load(f)

In [61]:
flat_pred, flat_gold = evaluate_predictions(onto_pred, ontonotes_main)

Map:   0%|          | 0/61371 [00:00<?, ? examples/s]

IndexError: list index out of range

In [55]:
from itertools import chain

# Flatten all token sequences into a single list
flat_tokens = list(chain.from_iterable(ontonotes_main['tokens']))
len(flat_tokens)

1110194

In [51]:
len(predictions[0])

4149

In [19]:
from collections import Counter
import pandas as pd


def compute_ner_distribution(dataset):
    # Flatten and filter out 'O'
    all_tags = [tag for example in dataset['ner_conll2003'] for tag in example]
    filtered_tags = [tag for tag in all_tags if tag != 'O']

    # Count tag frequencies
    tag_counts = Counter(filtered_tags)
    total = sum(tag_counts.values())

    # Create (tag, count, percentage) triples
    stats = sorted([(tag, count, count / total * 100)
                   for tag, count in tag_counts.items()])

    # Create DataFrame
    df = pd.DataFrame(stats, columns=["Tag", "Count", "Percentage"])
    return df


# Example usage
df = compute_ner_distribution(ontonotes_nw_encoded)
df2 = compute_ner_distribution(ontonotes_rest_encoded)
print('news')
print(df)
print('rest')
print(df2)

news
      Tag  Count  Percentage
0   B-LOC  16435   15.174738
1  B-MISC   7455    6.883339
2   B-ORG  22446   20.724805
3   B-PER  12641   11.671668
4   I-LOC   5154    4.758783
5  I-MISC   5579    5.151193
6   I-ORG  30834   28.469600
7   I-PER   7761    7.165874
rest
      Tag  Count  Percentage
0   B-LOC  14389   18.034718
1  B-MISC  10792   13.526352
2   B-ORG   7517    9.421570
3   B-PER  14691   18.413236
4   I-LOC   4350    5.452153
5  I-MISC   6920    8.673310
6   I-ORG   9826   12.315598
7   I-PER  11300   14.163063


# Load GPU

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Tokenisation & Alignment

In [24]:
ontonotes_id_to_label = {
    0: "O", 1: "B-CARDINAL", 2: "B-DATE", 3: "I-DATE", 4: "B-PERSON", 5: "I-PERSON",
    6: "B-NORP", 7: "B-GPE", 8: "I-GPE", 9: "B-LAW", 10: "I-LAW", 11: "B-ORG", 12: "I-ORG",
    13: "B-PERCENT", 14: "I-PERCENT", 15: "B-ORDINAL", 16: "B-MONEY", 17: "I-MONEY",
    18: "B-WORK_OF_ART", 19: "I-WORK_OF_ART", 20: "B-FAC", 21: "B-TIME", 22: "I-CARDINAL",
    23: "B-LOC", 24: "B-QUANTITY", 25: "I-QUANTITY", 26: "I-NORP", 27: "I-LOC",
    28: "B-PRODUCT", 29: "I-TIME", 30: "B-EVENT", 31: "I-EVENT", 32: "I-FAC",
    33: "B-LANGUAGE", 34: "I-PRODUCT", 35: "I-ORDINAL", 36: "I-LANGUAGE"
}

conll_label_to_id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3,
                     'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
id2label = {v: k for k, v in conll_label_to_id.items()}

ontonotes_to_conll_entity = {
    "PERSON": "PER", "ORG": "ORG", "GPE": "LOC", "LOC": "LOC",
    "NORP": "MISC", "FAC": "MISC", "EVENT": "MISC", "WORK_OF_ART": "MISC",
    "LAW": "MISC", "PRODUCT": "MISC", "LANGUAGE": "MISC",
    "DATE": None, "TIME": None, "PERCENT": None, "MONEY": None,
    "QUANTITY": None, "ORDINAL": None, "CARDINAL": None
}

In [56]:
def process_data(data_list):

    def process_single(data):
        word_ids = data['word_ids']
        predictions = data['predictions']
        gold = data['gold']
        tokenized_tokens = data['tokens']

        word_ids = [a for a in word_ids if a is not None]

        processed_predictions = []
        processed_gold = []

        current_word_id = None
        current_predictions = []
        current_gold = []

        for idx, word_id in enumerate(word_ids):
            if word_id != current_word_id:
                if current_predictions:
                    processed_predictions.append(
                        Counter(current_predictions).most_common(1)[0][0])
                    processed_gold.append(
                        Counter(current_gold).most_common(1)[0][0])

                current_word_id = word_id
                current_predictions = [predictions[idx]]
                current_gold = [gold[idx]]
            else:
                current_predictions.append(predictions[idx])
                current_gold.append(gold[idx])

        if current_predictions:
            processed_predictions.append(
                Counter(current_predictions).most_common(1)[0][0])
            processed_gold.append(
                Counter(current_gold).most_common(1)[0][0])

        return processed_predictions, processed_gold

    processed_predictions_list = []
    processed_gold_list = []

    for data in data_list:
        processed_predictions, processed_gold = process_single(data)
        processed_predictions_list.append(processed_predictions)
        processed_gold_list.append(processed_gold)

    return processed_predictions_list, processed_gold_list


def evaluate_predictions(p, test_data):
    predictions, labels, _ = p

    pred_indices = [np.argmax(p, axis=-1) for p in predictions]
    label_indices = labels

    pred_tags = [[id2label[p] for p, l in zip(p_seq, l_seq) if l != -100]
                 for p_seq, l_seq in zip(pred_indices, label_indices)]
    gold_tags = [[id2label[l] for l in l_seq if l != -100]
                 for l_seq in label_indices]

    def add_preds(example, idx):
        length = len(example['word_ids'])
        example['predictions'] = pred_tags[idx][:length]
        example['gold'] = gold_tags[idx][:length]
        return example

    test_data = test_data.map(add_preds, with_indices=True)

    length = len(test_data['predictions'][0])

    pred, gold = process_data(test_data)

    flat_pred = [label for seq in pred for label in seq]
    flat_gold = [label for seq in gold for label in seq]

    print(classification_report(flat_gold, flat_pred, zero_division=0))

    return (flat_pred, flat_gold)

# Model

In [59]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)

label_list = ['O', 'B-PER', 'I-PER', 'B-ORG',
              'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True,
        return_special_tokens_mask=True,
        return_offsets_mapping=True,
    )
    all_word_ids = []
    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        all_word_ids.append(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(labels[word_idx])
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    tokenized_inputs["word_ids"] = all_word_ids
    return tokenized_inputs

In [60]:
conll_main = conll_main.map(tokenize_and_align_labels, batched=True)
ontonotes_main = ontonotes_main.map(tokenize_and_align_labels, batched=True)
onto_news = ontonotes_nw_encoded.map(tokenize_and_align_labels, batched=True)
onto_rest = ontonotes_rest_encoded.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/16595 [00:00<?, ? examples/s]

Map:   0%|          | 0/61371 [00:00<?, ? examples/s]

Map:   0%|          | 0/42753 [00:00<?, ? examples/s]

# Deciding params

In [9]:
train_data = conll_main
test_data = onto_news

In [10]:
train_data_name = 'news'

In [11]:
mod = BertForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list))

metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)


data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir=f"./output/{model_name}",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True,
    save_strategy="no",
)

trainer = Trainer(
    model=mod,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/scratch/1063415/ipykernel_1294300/1248433719.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Step,Training Loss
500,0.248300
1000,0.089300
1500,0.055000
2000,0.050500
2500,0.033300
3000,0.028500


TrainOutput(global_step=3114, training_loss=0.08204985621960101, metrics={'train_runtime': 371.7057, 'train_samples_per_second': 133.937, 'train_steps_per_second': 8.378, 'total_flos': 3916666226042982.0, 'train_loss': 0.08204985621960101, 'epoch': 3.0})

In [13]:
trainer.save_model(f"./saved_model/{model_name}_{train_data_name}")

In [14]:
predictions = trainer.predict(test_data)

In [15]:
with open(f'./results/{model_name}_{train_data_name}_results.pkl', 'wb') as f:
    pickle.dump(predictions, f)

In [16]:
x, y = evaluate_predictions(predictions, test_data)

Map:   0%|          | 0/42753 [00:00<?, ? examples/s]

Map:   2%|▏         | 664/42753 [00:00<00:06, 6559.56 examples/s]

Map:   3%|▎         | 1342/42753 [00:00<00:08, 5096.73 examples/s]

Map:   5%|▍         | 2000/42753 [00:00<00:08, 4715.84 examples/s]

Map:   6%|▋         | 2723/42753 [00:00<00:07, 5505.22 examples/s]

...

Map:  97%|█████████▋| 41423/42753 [00:08<00:00, 4547.59 examples/s]

Map:  98%|█████████▊| 42000/42753 [00:08<00:00, 4625.95 examples/s]

Map: 100%|█████████▉| 42632/42753 [00:08<00:00, 5045.78 examples/s]

Map: 100%|██████████| 42753/42753 [00:09<00:00, 4401.07 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.83      0.91      0.87     16435
      B-MISC       0.52      0.73      0.60      7455
       B-ORG       0.72      0.76      0.74     22446
       B-PER       0.91      0.94      0.92     12641
       I-LOC       0.71      0.68      0.69      5154
      I-MISC       0.55      0.43      0.48      5579
       I-ORG       0.92      0.77      0.84     30834
       I-PER       0.91      0.92      0.92      7761
           O       0.99      0.99      0.99    965876

    accuracy                           0.97   1074181
   macro avg       0.78      0.79      0.78   1074181
weighted avg       0.97      0.97      0.97   1074181



In [17]:
predictions = trainer.predict(onto_rest)
x, y = evaluate_predictions(predictions, onto_rest)

Map:   0%|          | 0/100516 [00:00<?, ? examples/s]

Map:   1%|          | 766/100516 [00:00<00:13, 7546.51 examples/s]

Map:   2%|▏         | 1953/100516 [00:00<00:15, 6300.17 examples/s]

Map:   3%|▎         | 2862/100516 [00:00<00:15, 6165.79 examples/s]

Map:   4%|▍         | 3803/100516 [00:00<00:16, 5936.14 examples/s]

...

Map:  98%|█████████▊| 99000/100516 [00:19<00:00, 5587.67 examples/s]

Map:  99%|█████████▉| 99773/100516 [00:19<00:00, 6079.85 examples/s]

Map: 100%|██████████| 100516/100516 [00:20<00:00, 1224.34 examples/s]

Map: 100%|██████████| 100516/100516 [00:21<00:00, 4760.87 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.58      0.88      0.70     14389
      B-MISC       0.48      0.72      0.58     10792
       B-ORG       0.42      0.61      0.50      7517
       B-PER       0.45      0.90      0.60     14691
       I-LOC       0.54      0.66      0.59      4350
      I-MISC       0.44      0.39      0.41      6920
       I-ORG       0.70      0.62      0.66      9826
       I-PER       0.85      0.85      0.85     11300
           O       0.99      0.98      0.98   1568935

    accuracy                           0.96   1648720
   macro avg       0.61      0.73      0.65   1648720
weighted avg       0.97      0.96      0.97   1648720

